In [ ]:
import os
import sys
sys.path.append(os.path.dirname(os.path.realpath('__file__')) + '/../../../')
from acaisdk.file import File
from acaisdk.project import Project
from acaisdk.fileset import FileSet
from acaisdk.job import Job, JobStatus
from acaisdk.meta import *
from acaisdk.utils import utils
from acaisdk import credentials

workspace = os.path.dirname(os.path.realpath('__file__'))
utils.DEBUG = True  # print debug messages. Calls are successful as long as no exception is thrown.

In [ ]:
# Make your changes here
project_id = "AnotherAwesomeProject"
root_token = 'Ask sys guys for the root token'
project_admin = 'allen'
user = 'albert'

In [ ]:
# Create project and user
r = Project.create_project(project_id, root_token, project_admin)
r = Project.create_user(project_id, r['project_admin_token'], user)  # the new user is logged in automatically.

In [ ]:
# You can take note of the new token
token = r['user_token']
print(token)

# Next time you can use the token to login:
# credentials.login(token)

In [ ]:
# Upload code
train_code = os.path.join(workspace, 'demo/train_scripts.zip')
eval_code = os.path.join(workspace, 'demo/eval_script.zip')
File.upload({train_code: 'train_scripts.zip',
             eval_code: 'eval_script.zip'
            })

In [ ]:
# You can inspect the uploaded files
File.list_dir('/')

In [ ]:
# Upload input files and create a new file set on the fly
input_dir = os.path.join(workspace, 'demo/dataset/train/')
File.convert_to_file_mapping([input_dir], 'dataset/')[0]\
    .upload()\
    .as_new_file_set('train_dataset')

In [ ]:
# Run a training job, it takes ~3 mins to finish
job_setting = {
    "v_cpu": "0.5",
    "memory": "2000Mi",
    "gpu": "0",
    "command": "python3 train.py --output_folder ./my_output/train_out/ "
               "--train_data ./dataset/train/ "
               "--dev_data ./dataset/validation/ "
               "--epoch 10 "
               "--hidden 10 5",
    "container_image": "pytorch/pytorch",
    'input_file_set': 'train_dataset',
    'output_path': './my_output/train_out/',
    'code': 'train_scripts.zip',
    'description': 'train an image classification model',
    'name': 'my_acai_job'
}

train_job = Job().with_attributes(job_setting).register().run()

In [ ]:
# Wait until finish, in the mean time, you can login the 
# dashboard with your token and check the job status.

status = train_job.wait()
if status == JobStatus.FINISHED:
    output_file_set = train_job.output_file_set
    print("Job done. output file set id:", output_file_set)
else:
    print("Job went wrong:", status)

In [ ]:
# Take a look at what's in the output folder
File.list_dir('my_output/train_out/')

In [ ]:
input_dir = os.path.join(workspace, 'demo/dataset/eval')
File.convert_to_file_mapping([input_dir], 'dataset/eval/')[0]\
    .upload()\
    .as_new_file_set('eval_dataset')

In [ ]:
# Create a new file set that contains eval dataset and output model.
# Since all files we need are in their file set, we can simply create 
# the new file set out of the two existing ones.
FileSet.create_file_set('eval_data_and_model', ['@'+output_file_set, '@eval_dataset'])

In [ ]:
# ============= EVALUATION ==============
# Run a training job
job_setting_eval = {
    "v_cpu": "0.5",
    "memory": "500Mi",
    "gpu": "0",
    "command": "python3 eval.py --output_folder ./my_output/eval_out/ "
               "--eval_data ./dataset/eval/ "
               "--model ./my_output/train_out/network.npy ",
    "container_image": "pytorch/pytorch",
    'input_file_set': 'eval_data_and_model',
    'output_path': './my_output/eval_out/',
    'code': 'eval_script.zip',
    'description': 'evaluate an image classification model',
    'name': 'my_acai_job_2'
}

eval_job = Job().with_attributes(job_setting_eval).register().run()

In [ ]:
status = eval_job.wait()
if status == JobStatus.FINISHED:
    output_file_set = eval_job.output_file_set
    print("Job done. output file set id:", output_file_set)
else:
    print("Job went wrong:", status)

In [ ]:
# Download the result to local device
File.download({'/my_output/eval_out/eval_result.csv': workspace})

In [ ]:
# Check out the evaluation output
with open(os.path.join(workspace, 'eval_result.csv')) as f:
    for i, l in enumerate(f):
        print(l, end='')